# Feature Extraction - QSAR

_"Mathematically, feature extraction can be seen as a transformation of the original data space, into a new one described by fewer variables, usually orthogonal among them."_

## PCA

PCA is a dimensionality reduction technique that identifies important relationships in our data, transforms the existing data based on these relationships, and then quantifies the importance of these relationships so we can keep the most important relationships and drop the others. The main idea behind PCA is to find the best possible variables: the ones that summarize all the data as well as only possible, among all conceivable linear combinations of the original ones, and that simultaneously allow for reconstructing the original data, also as well as possible. To remember this definition, we can break it down into four steps:

* We identify the relationship among features through a Covariance Matrix.
* Through the linear transformation or eigendecomposition of the Covariance Matrix, we get eigenvectors and eigenvalues.
* Then we transform our data using Eigenvectors into principal components.
* Lastly, we quantify the importance of these relationships using Eigenvalues and keep the important principal components.


**Data Cleaning is Important**: PCA is sensitive to outliers and missing values. [source](https://towardsdatascience.com/feature-extraction-using-principal-component-analysis-a-simplified-visual-demo-e5592ced100a)

**Standardize Data**: PCA uses Euclidean distance as its feature vector similarity metric, so make sure we scale the features before applying PCA. [source](https://towardsdatascience.com/feature-extraction-using-principal-component-analysis-a-simplified-visual-demo-e5592ced100a)


That being said, Principal Component Analysis must **only** be performed on Data that has been standardized and have its outliers handled, since it is very sensitive to irregularities.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ds_functions as ds

import csv
import seaborn as sns

from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, IncrementalPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


#  “Covariance” indicates the direction of the linear relationship between variables.
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)

In [2]:
def feature_extraction_pca(X, y, filename):
    
    # Variance Explained
    mean = (X.mean(axis=0)).tolist()
    centered_data = X - mean
    cov_mtx = centered_data.cov()
    eigvals, eigvecs = np.linalg.eig(cov_mtx)

    pca = PCA()
    pca.fit(centered_data)
    PC = pca.components_
    var = pca.explained_variance_
    
    # Plot the covariance matrix for the dataset
    cov_mtx = np.round(cov_mtx, 3)
    sns.heatmap(cov_mtx, annot=True, fmt='g')
    plt.show()

    # PLOT EXPLAINED VARIANCE RATIO
    fig = plt.figure(figsize=(12, 6))
    plt.title('Explained variance ratio')
    plt.xlabel('PC')
    plt.ylabel('ratio')
    x_values = [str(i) for i in range(1, len(pca.components_) + 1)]
    bwidth = 0.5
    ax = plt.gca()
    ax.set_xticklabels(x_values)
    ax.set_ylim(0.0, 1.0)
    ax.bar(x_values, pca.explained_variance_ratio_, width=bwidth)
    ax.plot(pca.explained_variance_ratio_)
    for i, v in enumerate(pca.explained_variance_ratio_):
        ax.text(i, v+0.05, f'{v*100:.1f}', ha='center', fontweight='bold')
    plt.show()
    
    # Cumulative Variance Explained
    pca = PCA().fit(X)

    fig, ax = plt.subplots()
    xi = np.arange(1, len(pca.components_) + 1, step=1)
    yi = np.cumsum(pca.explained_variance_ratio_)

    plt.ylim(0.0,1.1)
    plt.plot(xi, yi, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, len(pca.components_) + 1, step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()
    
    # Alternatively, sklearn.decomposition.PCA supports a parameter that allows for the desired Cov threshold
    pca = PCA(n_components = 0.95).fit(X)

    # Transform the data according to the previous fit
    X_transformation = pca.transform(X)
    
    print(X_transformation.shape)
    
    # Merge results into a single dataframe
    y = np.array(y).reshape((np.array(y).shape[0], 1))
    to_be_df = np.append(X_transformation, y, axis=1)
    column_names = [("Eigen" + str(i)) for i in range (0, X_transformation.shape[1])] + ["DEATH_EVENT"]

    store_data = pd.DataFrame(data=to_be_df, columns=column_names)
    store_data.head()
    
    # Store DataFrame to a CSV
    output_file = '../datasets/pca_output/qsar/' + filename.split('/')[-1].split('.')[0] + '_pca.csv'
    store_data.to_csv(output_file, index=False)

In [3]:
def feature_extraction_kpca(X, y, filename):
    
    # Cumulative Variance Explained
    kpca = KernelPCA(kernel='rbf')

    kpca_transform = kpca.fit_transform(X)
    explained_variance = np.var(kpca_transform, axis=0)
    explained_variance_ratio = explained_variance / np.sum(explained_variance)

    fig, ax = plt.subplots()
    yi = np.cumsum(explained_variance_ratio)
    xi = np.arange(1, len(yi) + 1, step=1)

    plt.ylim(0.0,1.1)
    plt.plot(xi, yi, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, len(yi), step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()
    
    # Extract ideal number of components
    n_components = np.argmax(yi > 0.95) + 1
    
    # Alternatively, sklearn.decomposition.PCA supports a parameter that allows for the desired Cov threshold
    pca = KernelPCA(n_components=n_components, kernel='rbf').fit(X)

    # Transform the data according to the previous fit
    X_transformation = pca.transform(X)
    
    # Merge results into a single dataframe
    y = np.array(y).reshape((np.array(y).shape[0], 1))
    to_be_df = np.append(X_transformation, y, axis=1)
    column_names = [("Eigen" + str(i)) for i in range (0, X_transformation.shape[1])] + ["DEATH_EVENT"]

    store_data = pd.DataFrame(data=to_be_df, columns=column_names)
    store_data.head()
    
    # Store DataFrame to a CSV
    output_file = '../datasets/pca_output/qsar/' + filename.split('/')[-1].split('.')[0] + '_kpca.csv'
    store_data.to_csv(output_file, index=False)

In [4]:
def feature_extraction_ipca(X, y, filename):
    
    # Cumulative Variance Explained
    kpca = IncrementalPCA(batch_size=20)

    kpca_transform = kpca.fit_transform(X)
    explained_variance = np.var(kpca_transform, axis=0)
    explained_variance_ratio = explained_variance / np.sum(explained_variance)

    fig, ax = plt.subplots()
    yi = np.cumsum(explained_variance_ratio)
    xi = np.arange(1, len(yi) + 1, step=1)

    plt.ylim(0.0,1.1)
    plt.plot(xi, yi, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, len(yi), step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()
    
    # Extract ideal number of components
    n_components = np.argmax(yi > 0.95) + 1
    
    # Alternatively, sklearn.decomposition.PCA supports a parameter that allows for the desired Cov threshold
    pca = IncrementalPCA(n_components=n_components, batch_size=20).fit(X)

    # Transform the data according to the previous fit
    X_transformation = pca.transform(X)
    
    # Merge results into a single dataframe
    y = np.array(y).reshape((np.array(y).shape[0], 1))
    to_be_df = np.append(X_transformation, y, axis=1)
    column_names = [("Eigen" + str(i)) for i in range (0, X_transformation.shape[1])] + ["DEATH_EVENT"]

    store_data = pd.DataFrame(data=to_be_df, columns=column_names)
    store_data.head()
    
    # Store DataFrame to a CSV
    output_file = '../datasets/pca_output/qsar/' + filename.split('/')[-1].split('.')[0] + '_ipca.csv'
    store_data.to_csv(output_file, index=False)

In [5]:
def feature_extraction_lda(X, y, filename):
    
    lda = LDA(n_components=None)
    lda.fit(X, y)
    lda_var_ratios = lda.explained_variance_ratio_

    n_components = select_n_components(lda_var_ratios, 0.95)
    
    lda = LDA(n_components=n_components)
    X_transformation = lda.fit_transform(X, y)
    
    # Merge results into a single dataframe
    y = np.array(y).reshape((np.array(y).shape[0], 1))
    to_be_df = np.append(X_transformation, y, axis=1)
    column_names = [("Eigen" + str(i)) for i in range (0, X_transformation.shape[1])] + ["DEATH_EVENT"]

    store_data = pd.DataFrame(data=to_be_df, columns=column_names)
    store_data.head()
    
    output_file = '../datasets/pca_output/qsar/' + filename.split('/')[-1].split('.')[0] + '_lda.csv'
    store_data.to_csv(output_file, index=False)

In [6]:
def perform_component_analysis(filename):
    
    print("ANALYZING:", filename)
    
    data: pd.DataFrame = pd.read_csv(filename, sep=';', header=None)  
    
    y = np.array(data.pop(data.shape[1] - 1).values) # Target Variable
    y = [1 if i == 'positive' else 0 for i in y]

    X = pd.DataFrame(data.values) # Values of each feature on each record
    X = X.astype(np.int64)
    labels = pd.unique(y)
        
    feature_extraction_pca(X, y, filename)
    feature_extraction_ipca(X, y, filename)
    feature_extraction_lda(X, y, filename)


In [ ]:
import os

directory = '../datasets/pca_input/qsar'

overall_accs = []
datasets = []
    
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        path = directory + '/' + filename
        
        perform_component_analysis(path)

ANALYZING: ../datasets/pca_input/qsar/ORAL_anova.csv
